https://keras.io/preprocessing/image/

- 지난 시간에는 datagen.flow()를 통해 이미지가 generator를 지나게끔 해줬음
- 이번 시간에는 굳이 이미지를 open해서 바로 하는게 아니고 한번에 load해주는 걸 배워봄
    - 여러가지 방법 중, 제일 쉽고 간단한 방법은 폴더별로 레이블이 담겨있으면 그걸 바로 load하는 것
    - 따로 label 설정해줄 필요x 알아서 load해줌

# TensorFlow 2.0

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import os
from glob import glob

In [5]:
os.listdir('dataset/mnist_png/training')

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [1]:
# cifar 폴더는 mnist_png와 다르게 label이 파일명에 있음
# 그래서 이 방법 쓸 수 없고 좀 더 복잡(flow_from_dataframe)

In [12]:
train_dir = 'dataset/mnist_png/training'ng와 다르게 label이 파일명에 있음
# 그래서 이 방법 쓸 수 없고 좀 더 복잡(flow_from_dataframe)
test_dir = 'dataset/mnist_png/testing'

## Hyperparameter Tunning

In [7]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.7

input_shape = (28, 28, 1)
num_classes = 10

## Preprocess

In [13]:
train_datagen = ImageDataGenerator(
    rescale=1./255., # normalization
    width_shift_range=0.3,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale=1./255. # normalization
) # test는 agumentation말고 rescale만!

In [14]:
# 앞 강의에선 flow시키는 generetor 만들었지만 이번엔 다르게
train_generator = train_datagen.flow_from_directory(
    train_dir, # 경로
    target_size=input_shape[:2], # channel빼고 앞에 2개만 넣어줘도 됨
    batch_size=batch_size, # 이미지를 묶어서 학습에 넣는 것
    color_mode='grayscale', # mnist는 흑백이므로 gray scale로 넣어줘야 돼(default:RGB)
    class_mode='categorical' # 만약 binary라면 binary로 넣어줘야 돼
)

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical'
)

# class를 일일이 정해주지않아도 알아서 스스로 찾아냄!
# 물론, mnist_png같은 데이터에서만 가능하겠지(각 폴더별로 레이블)

# 혹시나 class가 12개인데 얘가 10개밖에 못 찾았다면
# parameter 중 classes=12 라고 해주면 됨

Found 60000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


## Build Model

In [15]:
inputs = layers.Input(input_shape)
net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [16]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

## Training

In [17]:
# 데이터를 불러서 모델에 학습하는 방법만 좀 다름
# 기존엔 model.fit()를 했었지
# 이번엔 generator를 따로 넣어줄 것이기때문에 fit_generator()사용
model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator), #몇 개의 batch_size(묶음)의 이미지가 epoch에서 다 돌면 그걸 1 epoch이라 할거냐 (안 넣어주면 error뜸)
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)


Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 1875 steps, validate for 313 steps
Epoch 1/10
 864/1875 [============>.................] - ETA: 3:50 - loss: 1.1401 - accuracy: 0.6049

KeyboardInterrupt: 